In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain, LLMChain

load_dotenv()

True

In [23]:
import neo4j

host = "localhost"
username = "neo4j"
password = "1234qwer"

# driver = neo4j.GraphDatabase.driver("neo4j://100.27.33.222:7687",
#   auth=neo4j.basic_auth("neo4j", "price-oxygens-scores")
# )
driver = neo4j.GraphDatabase.driver(f'bolt://localhost:7687', auth=(username, password))
session = driver.session()

def create_graph(query):
    session.run(query)

In [3]:
template1 = """You are expert english linguists 

Given sentence : $$$ {sentence} $$$

- Generate semantic triple that consist subject, predicate, and object  base on context sentence (which is delimited by $$$) 
- Output must separated by comma the format is [subject],[predicate],[object]
- Avoid using pronouns such as (you, he, she, it, we, they etc), replace with more relevant subject
- Each line is separated by '\n'
- Don't display anything in the output except the output csv format

"""

prompt1 = ChatPromptTemplate.from_template(template1)

model = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv('OPEN_API_KEY'), temperature=0)

chain1 = prompt1 | model

chain_one = LLMChain(llm=model, prompt=prompt1, output_key="triple")

overal_chain = SequentialChain(
    chains=[chain_one],
    input_variables=["sentence"],
    output_variables=["triple"]
)

/var/folders/80/kl6rpl3d2xj20wv4hyb0dbgc0000gn/T/ipykernel_12701/2577088822.py:19: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain_one = LLMChain(llm=model, prompt=prompt1, output_key="triple")


In [ ]:
import pandas as pd
import csv

df_dataset = pd.read_csv('output/datasets_all_sentence.csv')
for index, row in df_dataset.iterrows():
    sentence = row['sentence']
    app = row['app']
    uid = row['id']
    feature = row['feature']
    message = overal_chain({"sentence": sentence})
    triples = message['triple'].split('\n')

    for row in triples:
        with open("data/all_triplet_lang_result.csv", "a", newline="\n") as csvfile:
            writer = csv.writer(csvfile)
            row = f"{uid},{app},{feature},{row}"
            writer.writerow(row.split(","))

In [11]:
import pandas as pd
df_dataset = pd.read_csv('data/datasets_all_sentence.csv')

In [17]:
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

def preprocess_text(text, remove_stopwords=True, stem=False, lemmatize=True):
    """
    Preprocess text by applying various NLP techniques
    """
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    # Remove numbers
    text = ''.join(char for char in text if not char.isdigit())
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove stopwords
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
    
    # Initialize stemmer and lemmatizer
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    
    # Apply stemming or lemmatization
    if stem:
        tokens = [stemmer.stem(token) for token in tokens]
    elif lemmatize:
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join tokens back into text
    processed_text = ' '.join(tokens)
    
    return processed_text

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/fahmisyaifudin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/fahmisyaifudin/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fahmisyaifudin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/fahmisyaifudin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/fahmisyaifudin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [18]:
def df_to_create_cql(df, uid):
    cql_query = "MERGE (req:Requirement {id: '" + uid +"'}) \n"
    for index, row in df.iterrows():
        subject = preprocess_text(row['subject'])
        predicate = preprocess_text(row['predicate'])
        object = preprocess_text(row['object'])
        cql_query += f"MERGE (s{index}:CORE" + " {name: '" + subject + "'}) \n"
        cql_query += f"MERGE (p{index}:VERB" + " {name: '" + predicate + "'}) \n"
        cql_query += f"MERGE (o{index}:CORE" + " {name: '" + object + "'}) \n"
        cql_query += f"MERGE (s{index})-[:RELATED_TO]->(p{index})-[:RELATED_TO]->(o{index}) \n"
        cql_query += f"MERGE (req)-[:CONTAINS]->(s{index}) \n"
        cql_query += f"MERGE (req)-[:CONTAINS]->(p{index}) \n"
        cql_query += f"MERGE (req)-[:CONTAINS]->(o{index}) \n"
    return cql_query

In [24]:
import pandas as pd
df_result = pd.read_csv('data/all_triplet_lang_result.csv')

for index, row in df_dataset.iterrows():
    try:
        df_filter = df_result[df_result['id'] == row['id']]
        cql_query = df_to_create_cql(df_filter, row['id'])
        session.run(cql_query)
    except Exception as e:
        print(e)
        break

In [25]:
for index, row in df_dataset.iterrows():
    session.run(f"MATCH (r:Requirement) WHERE r.id = '{row['id']}' SET r.app = '{row['app']}', r.sentence = '{row['sentence'].replace("'", "")}'")

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

In [ ]:
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [ ]:
import pandas as pd

entities = []
records, summary, key = driver.execute_query("MATCH(r:Requirement)-[:CONTAINS]->(e) RETURN DISTINCT e.name")
for record in records:
    embedding = get_embedding(record['e.name'])
    entities.append({"entity": record['e.name'], "embedding": embedding})
df_entity = pd.DataFrame(entities)

In [ ]:
df_entity.to_csv('output/entity_llm_embedding.csv', index=False)

In [ ]:
import numpy as np
df_entity = pd.read_csv('output/entity_llm_embedding.csv')
df_entity['embedding'] = df_entity.embedding.apply(eval).apply(np.array)

In [ ]:
for index, row in df_entity.iterrows():
    session.run("MATCH (n:CORE|VERB) WHERE n.name = $name SET n.embedding = $embedding", name=row['entity'], embedding=row['embedding'])

### Mean Pooling

In [ ]:
import numpy as np

for index, row in df_dataset.iterrows():
    records, _, _ = driver.execute_query("MATCH (n:Requirement)-[:CONTAINS]->(target:CORE|VERB) WHERE n.id = $id RETURN target.embedding as embedding", id=row['id'])
    embeddings = []
    for record in records:
        embedding = record['embedding']
        embeddings.append(embedding)
    #aggregated_embeddings = [np.mean(embedding, axis=0) for embedding in embeddings]
    aggregated_embeddings = [np.max(embedding, axis=0) for embedding in embeddings]
    session.run("MATCH (n:Requirement) WHERE n.id = $id SET n.mean_embedding = $embedding", id=row['id'], embedding=aggregated_embeddings)